## `.BBoxTest.split_test` in Boston Housing price regression dataset

In [1]:
import numpy as np
import tensorflow.keras as keras 
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.python.keras import backend as K
import time
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam, SGD
from dnn_inference.BBoxTest import split_test
import tensorflow as tf


### Boston house prices dataset


**Data Set Characteristics:**  

    Number of Instances: 506 

    Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    Attribute Information (in order)
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per USD10,000
        - PTRATIO  pupil-teacher ratio by town
        - B        1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
        - LSTAT    % lower status of the population
        - MEDV     Median value of owner-occupied homes in $1000's


In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.boston_housing.load_data(path="boston_housing.npz", test_split=0.2)

In [3]:
n, d = x_train.shape
print('num of samples: %d, dim: %d' %(n, d))

num of samples: 404, dim: 13


In [10]:
from tensorflow import keras
from tensorflow.keras import layers

def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[d]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(1e-4)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [11]:
model_full, model_mask = build_model(), build_model()

In [12]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min',
					verbose=1, patience=10, restore_best_weights=True)

In [13]:
fit_params = {'callbacks': [es],
			  'epochs': 500,
			  'batch_size': 32,
			  'validation_split': .2,
			  'verbose': 2}

split_params = {'split': 'one-split',
				'perturb': None,
				'num_perm': 100,
				'ratio_grid': [.2, .4, .6, .8],
				'perturb_scale': 5,
				'min_inf': 0,
				'min_est': 0,
				'ratio_method': 'fuse',
				'cv_num': 1,
				'cp': 'min',
				'verbose': 1}

In [14]:
inf_feats = [
			np.arange(3),
			np.arange(5,11),
			]

In [15]:
cue = split_test(inf_feats=inf_feats,
			model=model_full, model_mask=model_mask, 
			change='mask', eva_metric='mse')

# - RAD(idx = 8) index of accessibility to radial highways
# is a cate feature.

p_value_tmp = cue.testing(x_train, y_train, cat_feats=[8],
							cv_num=3, cp='hommel', 
							fit_params=fit_params,
							split_params=split_params)

INFO:tensorflow:Assets written to: ./splitTest_checkpoints/model_init/assets
INFO:tensorflow:Assets written to: ./splitTest_checkpoints/model_mask_init/assets
Epoch 1/500
9/9 - 0s - loss: 1224.1019 - mae: 27.7544 - mse: 1224.1019 - val_loss: 738.0211 - val_mae: 20.3435 - val_mse: 738.0211 - 286ms/epoch - 32ms/step
Epoch 2/500
9/9 - 0s - loss: 570.5808 - mae: 17.5729 - mse: 570.5808 - val_loss: 365.7145 - val_mae: 13.9576 - val_mse: 365.7145 - 22ms/epoch - 2ms/step
Epoch 3/500
9/9 - 0s - loss: 320.5662 - mae: 12.7158 - mse: 320.5662 - val_loss: 232.7121 - val_mae: 11.6593 - val_mse: 232.7121 - 23ms/epoch - 3ms/step
Epoch 4/500
9/9 - 0s - loss: 213.0154 - mae: 10.6239 - mse: 213.0154 - val_loss: 171.1991 - val_mae: 10.0797 - val_mse: 171.1991 - 23ms/epoch - 3ms/step
Epoch 5/500
9/9 - 0s - loss: 155.2021 - mae: 9.0636 - mse: 155.2021 - val_loss: 126.3261 - val_mae: 8.2210 - val_mse: 126.3261 - 24ms/epoch - 3ms/step
Epoch 6/500
9/9 - 0s - loss: 116.8252 - mae: 7.7285 - mse: 116.8252 - val_

Epoch 56/500
9/9 - 0s - loss: 60.6443 - mae: 5.4151 - mse: 60.6443 - val_loss: 75.1848 - val_mae: 5.3127 - val_mse: 75.1848 - 22ms/epoch - 2ms/step
Epoch 57/500
9/9 - 0s - loss: 59.5892 - mae: 5.4697 - mse: 59.5892 - val_loss: 69.0603 - val_mae: 6.2525 - val_mse: 69.0603 - 22ms/epoch - 2ms/step
Epoch 58/500
9/9 - 0s - loss: 59.9599 - mae: 5.6996 - mse: 59.9599 - val_loss: 68.2425 - val_mae: 6.1698 - val_mse: 68.2425 - 23ms/epoch - 3ms/step
Epoch 59/500
9/9 - 0s - loss: 59.3036 - mae: 5.7686 - mse: 59.3036 - val_loss: 67.8119 - val_mae: 5.4330 - val_mse: 67.8119 - 23ms/epoch - 3ms/step
Epoch 60/500
9/9 - 0s - loss: 58.1098 - mae: 5.4721 - mse: 58.1098 - val_loss: 69.8784 - val_mae: 5.3231 - val_mse: 69.8784 - 23ms/epoch - 3ms/step
Epoch 61/500
9/9 - 0s - loss: 57.5600 - mae: 5.4390 - mse: 57.5600 - val_loss: 73.2949 - val_mae: 5.2798 - val_mse: 73.2949 - 22ms/epoch - 2ms/step
Epoch 62/500
9/9 - 0s - loss: 57.3872 - mae: 5.3094 - mse: 57.3872 - val_loss: 68.3963 - val_mae: 5.3951 - val_m

Epoch 112/500
9/9 - 0s - loss: 50.6747 - mae: 5.1600 - mse: 50.6747 - val_loss: 60.3867 - val_mae: 5.2557 - val_mse: 60.3867 - 24ms/epoch - 3ms/step
Epoch 113/500
9/9 - 0s - loss: 51.7009 - mae: 5.1679 - mse: 51.7009 - val_loss: 69.4086 - val_mae: 4.9401 - val_mse: 69.4086 - 22ms/epoch - 2ms/step
Epoch 114/500
9/9 - 0s - loss: 50.7615 - mae: 4.8891 - mse: 50.7615 - val_loss: 60.2344 - val_mae: 5.1560 - val_mse: 60.2344 - 23ms/epoch - 3ms/step
Epoch 115/500
9/9 - 0s - loss: 50.4772 - mae: 5.1873 - mse: 50.4772 - val_loss: 92.7796 - val_mae: 6.2204 - val_mse: 92.7796 - 22ms/epoch - 2ms/step
Epoch 116/500
9/9 - 0s - loss: 51.7434 - mae: 5.0604 - mse: 51.7434 - val_loss: 75.6717 - val_mae: 5.2303 - val_mse: 75.6717 - 21ms/epoch - 2ms/step
Epoch 117/500
9/9 - 0s - loss: 50.7705 - mae: 4.9086 - mse: 50.7705 - val_loss: 60.1377 - val_mae: 5.4792 - val_mse: 60.1377 - 23ms/epoch - 3ms/step
Epoch 118/500
9/9 - 0s - loss: 50.9858 - mae: 5.2424 - mse: 50.9858 - val_loss: 73.5796 - val_mae: 5.1087 

Epoch 4/500
9/9 - 0s - loss: 242.6159 - mae: 11.5458 - mse: 242.6159 - val_loss: 189.8232 - val_mae: 10.7264 - val_mse: 189.8232 - 22ms/epoch - 2ms/step
Epoch 5/500
9/9 - 0s - loss: 178.3219 - mae: 9.8691 - mse: 178.3219 - val_loss: 131.2843 - val_mae: 8.7064 - val_mse: 131.2843 - 23ms/epoch - 3ms/step
Epoch 6/500
9/9 - 0s - loss: 128.6482 - mae: 8.4035 - mse: 128.6482 - val_loss: 105.2626 - val_mae: 7.4501 - val_mse: 105.2626 - 23ms/epoch - 3ms/step
Epoch 7/500
9/9 - 0s - loss: 103.6493 - mae: 7.4510 - mse: 103.6493 - val_loss: 93.5494 - val_mae: 6.6470 - val_mse: 93.5494 - 22ms/epoch - 2ms/step
Epoch 8/500
9/9 - 0s - loss: 90.1052 - mae: 6.9328 - mse: 90.1052 - val_loss: 95.9410 - val_mae: 6.5007 - val_mse: 95.9410 - 21ms/epoch - 2ms/step
Epoch 9/500
9/9 - 0s - loss: 85.6111 - mae: 6.5784 - mse: 85.6111 - val_loss: 81.0128 - val_mae: 7.0444 - val_mse: 81.0128 - 21ms/epoch - 2ms/step
Epoch 10/500
9/9 - 0s - loss: 81.7400 - mae: 6.9944 - mse: 81.7400 - val_loss: 80.6307 - val_mae: 5.97

Epoch 60/500
9/9 - 0s - loss: 56.1880 - mae: 5.1750 - mse: 56.1880 - val_loss: 60.4176 - val_mae: 4.8747 - val_mse: 60.4176 - 23ms/epoch - 3ms/step
Epoch 61/500
9/9 - 0s - loss: 53.8901 - mae: 5.0470 - mse: 53.8901 - val_loss: 54.6698 - val_mae: 5.3438 - val_mse: 54.6698 - 22ms/epoch - 2ms/step
Epoch 62/500
9/9 - 0s - loss: 54.5774 - mae: 5.2102 - mse: 54.5774 - val_loss: 61.6746 - val_mae: 4.7714 - val_mse: 61.6746 - 23ms/epoch - 3ms/step
Epoch 63/500
9/9 - 0s - loss: 53.5169 - mae: 5.0393 - mse: 53.5169 - val_loss: 70.9629 - val_mae: 5.1392 - val_mse: 70.9629 - 22ms/epoch - 2ms/step
Epoch 64/500
9/9 - 0s - loss: 54.0228 - mae: 5.0509 - mse: 54.0228 - val_loss: 79.3993 - val_mae: 5.6210 - val_mse: 79.3993 - 22ms/epoch - 2ms/step
Epoch 65/500
9/9 - 0s - loss: 55.3411 - mae: 5.0911 - mse: 55.3411 - val_loss: 54.4621 - val_mae: 4.8212 - val_mse: 54.4621 - 23ms/epoch - 3ms/step
Epoch 66/500
9/9 - 0s - loss: 53.7104 - mae: 5.0589 - mse: 53.7104 - val_loss: 66.1331 - val_mae: 4.9300 - val_m

...

9/9 - 0s - loss: 45.7927 - mae: 4.6062 - mse: 45.7927 - val_loss: 26.4946 - val_mae: 3.5296 - val_mse: 26.4946 - 23ms/epoch - 3ms/step
Epoch 141/500
9/9 - 0s - loss: 46.5955 - mae: 4.7052 - mse: 46.5955 - val_loss: 37.9537 - val_mae: 5.0986 - val_mse: 37.9537 - 21ms/epoch - 2ms/step
Epoch 142/500
9/9 - 0s - loss: 45.3616 - mae: 4.8506 - mse: 45.3616 - val_loss: 26.7167 - val_mae: 3.6078 - val_mse: 26.7167 - 22ms/epoch - 2ms/step
Epoch 143/500
9/9 - 0s - loss: 45.0352 - mae: 4.5912 - mse: 45.0352 - val_loss: 26.3731 - val_mae: 3.4586 - val_mse: 26.3731 - 22ms/epoch - 2ms/step
Epoch 144/500
9/9 - 0s - loss: 45.3538 - mae: 4.5526 - mse: 45.3538 - val_loss: 29.4527 - val_mae: 4.1186 - val_mse: 29.4527 - 21ms/epoch - 2ms/step
Epoch 145/500
9/9 - 0s - loss: 44.7032 - mae: 4.5960 - mse: 44.7032 - val_loss: 46.8655 - val_mae: 5.8629 - val_mse: 46.8655 - 22ms/epoch - 2ms/step
Epoch 146/500
9/9 - 0s - loss: 45.7508 - mae: 4.8376 - mse: 45.7508 - val_loss: 26.6843 - val_mae: 3.6722 - val_mse: 26.

Epoch 41/500
9/9 - 0s - loss: 60.4327 - mae: 5.5104 - mse: 60.4327 - val_loss: 49.0680 - val_mae: 5.3268 - val_mse: 49.0680 - 24ms/epoch - 3ms/step
Epoch 42/500
9/9 - 0s - loss: 59.6724 - mae: 5.4175 - mse: 59.6724 - val_loss: 50.1001 - val_mae: 5.0419 - val_mse: 50.1001 - 22ms/epoch - 2ms/step
Epoch 43/500
9/9 - 0s - loss: 60.8938 - mae: 5.3943 - mse: 60.8938 - val_loss: 53.9293 - val_mae: 5.0413 - val_mse: 53.9293 - 23ms/epoch - 3ms/step
Epoch 44/500
9/9 - 0s - loss: 61.5622 - mae: 5.3955 - mse: 61.5622 - val_loss: 49.9671 - val_mae: 5.3839 - val_mse: 49.9671 - 22ms/epoch - 2ms/step
Epoch 45/500
9/9 - 0s - loss: 59.6289 - mae: 5.5427 - mse: 59.6289 - val_loss: 52.3975 - val_mae: 5.5642 - val_mse: 52.3975 - 23ms/epoch - 3ms/step
Epoch 46/500
9/9 - 0s - loss: 59.3820 - mae: 5.4686 - mse: 59.3820 - val_loss: 51.7482 - val_mae: 4.9884 - val_mse: 51.7482 - 22ms/epoch - 2ms/step
Epoch 47/500
9/9 - 0s - loss: 60.0178 - mae: 5.3841 - mse: 60.0178 - val_loss: 50.1365 - val_mae: 4.9702 - val_m

Epoch 97/500
9/9 - 0s - loss: 50.5468 - mae: 4.9600 - mse: 50.5468 - val_loss: 64.9190 - val_mae: 7.0824 - val_mse: 64.9190 - 22ms/epoch - 2ms/step
Epoch 98/500
9/9 - 0s - loss: 52.7750 - mae: 5.4421 - mse: 52.7750 - val_loss: 38.8132 - val_mae: 4.5826 - val_mse: 38.8132 - 25ms/epoch - 3ms/step
Epoch 99/500
9/9 - 0s - loss: 50.0207 - mae: 4.9085 - mse: 50.0207 - val_loss: 38.6494 - val_mae: 4.4190 - val_mse: 38.6494 - 22ms/epoch - 2ms/step
Epoch 100/500
9/9 - 0s - loss: 50.1258 - mae: 4.8783 - mse: 50.1258 - val_loss: 42.4284 - val_mae: 4.4198 - val_mse: 42.4284 - 23ms/epoch - 3ms/step
Epoch 101/500
9/9 - 0s - loss: 49.5411 - mae: 4.8601 - mse: 49.5411 - val_loss: 39.8443 - val_mae: 4.9512 - val_mse: 39.8443 - 22ms/epoch - 2ms/step
Epoch 102/500
9/9 - 0s - loss: 49.3007 - mae: 4.8861 - mse: 49.3007 - val_loss: 44.6216 - val_mae: 5.5425 - val_mse: 44.6216 - 24ms/epoch - 3ms/step
Epoch 103/500
9/9 - 0s - loss: 50.1141 - mae: 5.0370 - mse: 50.1141 - val_loss: 39.6915 - val_mae: 4.9155 - v

Epoch 37/500
9/9 - 0s - loss: 60.0494 - mae: 5.3100 - mse: 60.0494 - val_loss: 43.5436 - val_mae: 4.3302 - val_mse: 43.5436 - 23ms/epoch - 3ms/step
Epoch 38/500
9/9 - 0s - loss: 57.9962 - mae: 5.1360 - mse: 57.9962 - val_loss: 43.3393 - val_mae: 4.6667 - val_mse: 43.3393 - 23ms/epoch - 3ms/step
Epoch 39/500
9/9 - 0s - loss: 58.2837 - mae: 5.2399 - mse: 58.2837 - val_loss: 42.7238 - val_mae: 4.3220 - val_mse: 42.7238 - 23ms/epoch - 3ms/step
Epoch 40/500
9/9 - 0s - loss: 58.0304 - mae: 5.2399 - mse: 58.0304 - val_loss: 43.3122 - val_mae: 4.2196 - val_mse: 43.3122 - 22ms/epoch - 2ms/step
Epoch 41/500
9/9 - 0s - loss: 58.8655 - mae: 5.1446 - mse: 58.8655 - val_loss: 59.0361 - val_mae: 6.5785 - val_mse: 59.0361 - 22ms/epoch - 2ms/step
Epoch 42/500
9/9 - 0s - loss: 60.4698 - mae: 5.7397 - mse: 60.4698 - val_loss: 42.3079 - val_mae: 4.7172 - val_mse: 42.3079 - 23ms/epoch - 3ms/step
Epoch 43/500
9/9 - 0s - loss: 57.4504 - mae: 5.3325 - mse: 57.4504 - val_loss: 44.2561 - val_mae: 4.2126 - val_m

Epoch 93/500
9/9 - 0s - loss: 49.2957 - mae: 4.9568 - mse: 49.2957 - val_loss: 35.4323 - val_mae: 3.7607 - val_mse: 35.4323 - 22ms/epoch - 2ms/step
Epoch 94/500
9/9 - 0s - loss: 50.0401 - mae: 4.6051 - mse: 50.0401 - val_loss: 32.9133 - val_mae: 4.0198 - val_mse: 32.9133 - 23ms/epoch - 3ms/step
Epoch 95/500
9/9 - 0s - loss: 48.6952 - mae: 4.7925 - mse: 48.6952 - val_loss: 33.0879 - val_mae: 3.9282 - val_mse: 33.0879 - 21ms/epoch - 2ms/step
Epoch 96/500
9/9 - 0s - loss: 48.3793 - mae: 4.6466 - mse: 48.3793 - val_loss: 35.6564 - val_mae: 4.7154 - val_mse: 35.6564 - 22ms/epoch - 2ms/step
Epoch 97/500
9/9 - 0s - loss: 49.5149 - mae: 4.8269 - mse: 49.5149 - val_loss: 33.1895 - val_mae: 3.7682 - val_mse: 33.1895 - 21ms/epoch - 2ms/step
Epoch 98/500
9/9 - 0s - loss: 49.1868 - mae: 4.8046 - mse: 49.1868 - val_loss: 33.3033 - val_mae: 3.7534 - val_mse: 33.3033 - 23ms/epoch - 3ms/step
Epoch 99/500
9/9 - 0s - loss: 48.4719 - mae: 4.6926 - mse: 48.4719 - val_loss: 33.1123 - val_mae: 3.7709 - val_m

In [16]:
p_value_tmp

[1.0, 1.253132091567549e-12]